
# Data Preprocessing
In this notebook we load and process the raw data to develop the final dataset for the IBM-Z Datathon. We make use of two main datasets for a list of all observed geoeffective CMEs from the post-SOHO era between 1996-2024, and two data sets for features and targets:

#### Geo-effective CMEs:
- The [Richardson and Cane list](https://izw1.caltech.edu/ACE/ASC/DATA/level3/icmetable2.htm); a list of near-Earth CMEs from 1996-2024.
- The [George Mason University CME/ICME list](http://solar.gmu.edu/heliophysics/index.php/GMU_CME/ICME_List); a list of geoeffective CMEs from 2007-2017.

#### Features and Targets:
- The [SOHO-LASCO CME Catalogue](https://cdaw.gsfc.nasa.gov/CME_list/); a list of all CMEs observed from 1996-2024 containing information on physical quantities.
- [OMNIWeb Plus data](https://omniweb.gsfc.nasa.gov/); a list of features associated with the solar wind and sunspot numbers.


## Cleaning the data:

In [64]:
# Importing libraries:
# For data manipulation
import pandas as pd

#For data visualisation:
import matplotlib.pyplot as plt


#### SOHO-LASCO Catalogue

We begin by loading in the SOHO-LASCO Catalogue to obtain the physical quantities for all CMEs. The original dataset had 11 total features. Most of the data was missing for the mass and kinetic energy hence these have been excluded. We have also excluded the second-order speeds as these are correlated with the linear speed. As a result this dataset contains the dates and times for each CME, together with five features:
- Central Position Angle in degrees.
- Angular Width in degrees.
- Linear Speed in km/s.
- Acceleration in km/s$^2$.
- Measurement Position Angle in degrees.


In [65]:
# Adding filepaths as variables
cane_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/RichardsonCane.csv"
gmu_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/GMU.csv"
soho_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/SOHO_LASCO.csv"
omniweb_file_path = r"/Users/yusufmspahi/PycharmProjects/IBM-Z_Datathon_2024/data/OMNIWeb.csv"

In [66]:
# Reading SOHO-LASCO dataset
soho_df = pd.read_csv(soho_file_path)
soho_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39285 entries, 0 to 39284
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          39285 non-null  object 
 1   Time          39285 non-null  object 
 2   CentralPA     39285 non-null  object 
 3   AngularWidth  39285 non-null  int64  
 4   LinearSpeed   39166 non-null  float64
 5   Accel         39285 non-null  object 
 6   MPA           39285 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.1+ MB


In [67]:
soho_df.head(200)

,Date,Time,CentralPA,AngularWidth,LinearSpeed,Accel,MPA
0,11/01/1996,00:14:36,267,18,499.0,-64.3*,272
1,13/01/1996,22:08:30,265,16,290.0,2.8*,266
2,15/01/1996,07:01:10,262,43,525.0,-31.1,272
3,22/01/1996,03:11:01,105,37,267.0,-126.3*,103
4,26/01/1996,09:16:19,90,27,262.0,1.9*,90
...,...,...,...,...,...,...,...
195,20/06/1996,13:53:43,73,16,220.0,0.8*,82
196,20/06/1996,17:49:03,105,38,80.0,6.6*,103
197,20/06/1996,19:18:00,1,6,91.0,-4.3*,360
198,21/06/1996,02:07:30,260,20,154.0,2.2*,265


After inspecting the dataset, we will do the following:
- Convert all missing values labelled as "------" and "NaN" to `None`.
- Convert Central PA values labelled as "Halo" to 360.
- Reformat the Acceleration column by removing asterisks.
- Convert all columns to numeric.
- Remove CME data corresponding to an angular width below 90 degrees as it is known that these are not likely to be geoeffective.
- Replace labelled columns "Date" and "Time" with a single "Datetime" column.

In [68]:
# Step 1: Replace all missing values ('------' and 'NaN') with None
soho_df.replace(['------', 'NaN'], None, inplace=True)

# Step 2: Convert Angular Width values labelled as "Halo" to 360
soho_df['CentralPA'] = soho_df['CentralPA'].replace('Halo', 360)

# Step 3: Remove asterisks from the Acceleration column
soho_df['Accel'] = soho_df['Accel'].astype(str).str.replace('*', '', regex=False)

# Step 4: Convert all columns to numeric, except the first two (Date and Time)
cols_to_convert = soho_df.columns[2:]  # Keep first two columns (Date and Time) as object
soho_df[cols_to_convert] = soho_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Step 5: Remove rows where Central PA is below 90 degrees
soho_df = soho_df[soho_df['AngularWidth'] >= 90]

# Step 6: Combine first two columns
soho_df['Datetime'] = pd.to_datetime(soho_df['Date'] + ' ' + soho_df['Time'], dayfirst=True) # Creating Datetime column
soho_df = soho_df.drop(soho_df.columns[[0, 1]], axis=1) # Drop the first two columns by index
last_column = soho_df.pop(soho_df.columns[-1])  # Pop the last column
soho_df.insert(0, last_column.name, last_column)  # Insert it at the front

soho_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5873 entries, 12 to 39282
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Datetime      5873 non-null   datetime64[ns]
 1   CentralPA     5873 non-null   int64         
 2   AngularWidth  5873 non-null   int64         
 3   LinearSpeed   5867 non-null   float64       
 4   Accel         5851 non-null   float64       
 5   MPA           5873 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 321.2 KB


In [69]:
soho_df.head(200) 

,Datetime,CentralPA,AngularWidth,LinearSpeed,Accel,MPA
12,1996-02-02 23:00:47,180,119,80.0,1.8,164
83,1996-04-29 14:38:48,360,360,65.0,NaN,149
85,1996-05-01 08:41:46,94,95,314.0,0.7,70
188,1996-06-18 17:28:50,84,95,64.0,-0.4,79
285,1996-07-20 09:28:16,31,175,246.0,9.4,34
...,...,...,...,...,...,...
2561,1998-11-07 20:54:05,321,96,750.0,23.7,314
2565,1998-11-08 11:54:05,264,196,559.0,6.2,214
2569,1998-11-09 01:54:05,16,94,144.0,0.7,27
2573,1998-11-09 18:17:55,330,190,325.0,2.6,338


#### OMNIWeb Plus Dataset
To obtain the final list of features, we must concatenate the SOHO-LASCO Catalogue with the OMNIWeb Plus Dataset. This dataset contains 12 features associated with the solar wind:

- The X-component of the magnetic field in nT.
- The Y-component of the magnetic field in nT.
- The Z-component of the magnetic field in nT.
- Plasma Temperature in Kelvin.
- Solar Proton Density n/cc.
- Flow Speed in km/s.
- Longitude Angle in degrees.
- Latitude Angle in degrees.
- Proton Density Ratio (unitless).
- Flow Pressure in nPa.
- Plasma Beta (unitless).
- Sunspot Number.

In [70]:
# Loading in OMNIWeb Plus data
omniweb_df = pd.read_csv(omniweb_file_path)
omniweb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261168 entries, 0 to 261167
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Datetime           261168 non-null  object 
 1   BX                 261168 non-null  float64
 2   BY                 261168 non-null  float64
 3   BZ                 261168 non-null  float64
 4   Plasma_Temp        261168 non-null  int64  
 5   Proton_Density     261168 non-null  float64
 6   Plasma_Speed       261168 non-null  int64  
 7   Plasma_Long_Angle  261168 non-null  float64
 8   Plasma_Lat_Angle   261168 non-null  float64
 9   Alpha_Prot_Ratio   261168 non-null  float64
 10  Flow_Pressure      261168 non-null  float64
 11  Plasma_Beta        261168 non-null  float64
 12  Sunspot_No         261168 non-null  int64  
dtypes: float64(9), int64(3), object(1)
memory usage: 25.9+ MB


In [71]:
omniweb_df.head()

,Datetime,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
0,01/01/1995 00:00,-0.6,3.2,-2.1,15816,16.3,315,-0.4,4.6,0.016,2.87,6.11,13
1,01/01/1995 01:00,-0.9,2.4,-1.3,15601,18.8,315,-0.1,3.1,0.013,3.28,12.51,13
2,01/01/1995 02:00,0.5,1.8,-0.4,20703,19.3,320,0.3,2.1,0.012,3.46,11.69,13
3,01/01/1995 03:00,-3.2,2.5,0.3,25094,16.4,317,1.6,2.2,0.013,2.90,5.66,13
4,01/01/1995 04:00,-4.3,2.1,0.3,25604,14.5,313,1.7,2.2,0.015,2.51,4.03,13


After taking a look at the OMNIWeb Plus Data, we next to the following to the dataset:
- Reformat first column to match the SOHO-LASCO data.
- Match times by averaging the 6-hour window after CME ejection.
- Concatenate both datasets to obtain final list of all 17 features.

In [72]:
# Step 1: Reformat Datetime column
omniweb_df['Datetime'] = pd.to_datetime(omniweb_df['Datetime'] + ':00', dayfirst=True)
omniweb_df.head()

,Datetime,BX,BY,BZ,Plasma_Temp,Proton_Density,Plasma_Speed,Plasma_Long_Angle,Plasma_Lat_Angle,Alpha_Prot_Ratio,Flow_Pressure,Plasma_Beta,Sunspot_No
0,1995-01-01 00:00:00,-0.6,3.2,-2.1,15816,16.3,315,-0.4,4.6,0.016,2.87,6.11,13
1,1995-01-01 01:00:00,-0.9,2.4,-1.3,15601,18.8,315,-0.1,3.1,0.013,3.28,12.51,13
2,1995-01-01 02:00:00,0.5,1.8,-0.4,20703,19.3,320,0.3,2.1,0.012,3.46,11.69,13
3,1995-01-01 03:00:00,-3.2,2.5,0.3,25094,16.4,317,1.6,2.2,0.013,2.90,5.66,13
4,1995-01-01 04:00:00,-4.3,2.1,0.3,25604,14.5,313,1.7,2.2,0.015,2.51,4.03,13
